<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
#!pip install -U spacy
#!python -m spacy download en_core_web_lg 
#!python -m spacy download en

Устанавливаем библиотеки (2 нижние строчки не обязательны, но у меня без них не работало)

In [2]:
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

import spacy
import re
import nltk

from sklearn.utils import resample
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

Импортируем нужные инструменты

In [3]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv('/home/brshk-shaun/Документы/datasets/toxic_comments.csv')
finally:
    display(data.info(), data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Загружаем датасет и смотрим

In [4]:
display(len(data[data['toxic'] == 1]), len(data[data['toxic'] == 0]))

toxic_sample = data[data['toxic'] == 1]
untoxic_sample = data[data['toxic'] == 0]

16186

143106

В датасете обнаружен дисбаланс классов

In [6]:
disabled_pipes = ['parser',  'ner']
spcy = spacy.load('en_core_web_sm',disable=disabled_pipes)

def lemmatize(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    text = text.split()
    
    filtered = spcy(' '.join(text))
    lemma = [y.lemma_ for y in filtered]
    
    final = ' '.join(lemma)
    
    return final

data['text_new'] = data['text'].progress_apply(lambda x: lemmatize(x))

  0%|          | 0/159292 [00:00<?, ?it/s]

Так как фичи загрязнены, пишем функцию для очистки текста от знаков переноса и лишних символов. В этой же функции применяем лемматизацию на основе **Spacy**. Лемматизированные признаки добавим в новый столбец датафрейма, на всякий случай сохранив основной текст.

In [7]:
features = data['text_new']
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=666
)

display(target_train.value_counts(), target_test.value_counts())

0    128791
1     14571
Name: toxic, dtype: int64

0    14315
1     1615
Name: toxic, dtype: int64

In [8]:
stopwords = nltk.corpus.stopwords.words("english")

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

Делим датасет на выборки и вызываем конвертер строк для дальнейшего его применения в пайплайне.

## Обучение

In [11]:
logreg = LogisticRegression()
logreg_pipe = Pipeline([('vectorize', count_tf_idf), ('logreg', logreg)])


params_log = {
    'logreg__random_state': [666],
    'logreg__solver': ['liblinear'],
    'logreg__C': range(5, 15)
}

logreg_grid = GridSearchCV(logreg_pipe, param_grid=params_log, cv=3, scoring='f1')
%time logreg_grid.fit(features_train, target_train)

print('Best score: {}'.format(logreg_grid.best_score_))

CPU times: user 6min 29s, sys: 3min 46s, total: 10min 16s
Wall time: 10min 18s
Best score: 0.778371586320071


С помощью пайплайна передаем в гридсерч **логической регрессии** подготовленные конвертером фичи.
При времени обучения **5 min 32s.** результат на тренировочной выборке составил **0.77**.

In [12]:
lgb_model = lgb.LGBMClassifier(random_state=666)
lgb_pipe = Pipeline([('vectorize', count_tf_idf), ('lgb', lgb_model)])

params_lgb = {
    'lgb__boosting_type': ['gbdt', 'dart'],
    'lgb__objective': ['binary']
}

lgb_grid = GridSearchCV(lgb_pipe, params_lgb, cv=3, scoring='f1')
%time lgb_grid.fit(features_train, target_train)

print('Best score: {}'.format(lgb_grid.best_score_))

CPU times: user 14min 50s, sys: 0 ns, total: 14min 50s
Wall time: 14min 53s
Best score: 0.7542186152246199


Также поступаем с **LightGBM**. Время обучения составило **16 минут 57 секунд**, при этом результат хуже - **0.75**.

## Выводы

In [13]:
logreg_final_pred = logreg_grid.best_estimator_.predict(features_test)

print('F1 score: {}'.format(f1_score(target_test, logreg_final_pred)))

F1 score: 0.781515460414543


Так как логическая регрессия оказалась лучше, тестовую выборку проверяем на ней. Финальный результат составил **0.78**, что довольно не плохо. Также стоит отметить, что логическая регрессия очень быстрая, поэтому даже на больших выборках вычисления не будут занимать много времени.